# ETL - COVID State Vaccination Data
This project consolidates COVID vaccination data for the US by State to prepare it for uploading to a Postgres Database 
- Notes on the data: Unfortunately, I was not able to find consolidated vaccine data on the CDC websites that would give me everything that I wanted.....I could get culumulative totals by state, and I could get daily counts at the national level, but I wanted daily vaccination counts by State. So I had to useOur World in Data's site and download several different files to get the data that I wanted. https://ourworldindata.org/us-states-vaccinations
    - us-covid-number-fully-vaccinated-in-US.csv
    - us-covid-share-fully-vaccinated.csv
    - us-daily-covid-vaccine-doses-administered-by-state.csv
    - us-daily-covid-vaccine-doses-per-million.csv

The various steps are detailed below:
1. [Merge CSVs of State-level Vaccination Data](#Merge-CSVs-of-State-level-Vaccination-Data ) 


In [1]:
# Dependencies
import pandas as pd


## Extract, Merge and Clean State level Vaccination Data 

In [2]:
# Files to Load
number_fully_vaccinated_to_load = "Resources/us-covid-number-fully-vaccinated-in-US.csv"
share_fully_vaccinated_to_load = "Resources/us-covid-share-fully-vaccinated.csv"
number_doses_administered_to_load = "Resources/us-daily-covid-vaccine-doses-administered-by-state.csv"
number_doses_per_million_to_load = "Resources/us-daily-covid-vaccine-doses-per-million.csv"

# Read Vaccine date files and store into Pandas DataFrames
nbr_fully_vaccinated_df = pd.read_csv(number_fully_vaccinated_to_load)
shr_fully_vaccinated_df = pd.read_csv(share_fully_vaccinated_to_load)
nbr_doses_administered_df = pd.read_csv(number_doses_administered_to_load)
nbr_doses_per_million_df = pd.read_csv(number_doses_per_million_to_load)


In [3]:
# Remove empty columns before merging
nbr_fully_vaccinated_df = nbr_fully_vaccinated_df.drop(columns=['Code'])
shr_fully_vaccinated_df = shr_fully_vaccinated_df.drop(columns=['Code'])
nbr_doses_administered_df = nbr_doses_administered_df.drop(columns=['Code'])
nbr_doses_per_million_df = nbr_doses_per_million_df.drop(columns=['Code'])

# Combine the data into a single dataset dropping the empty columns 
df1 = pd.merge(nbr_fully_vaccinated_df, shr_fully_vaccinated_df, how="left", on=["Entity","Date"])
df2 = pd.merge(df1,nbr_doses_administered_df, how="left", on=["Entity","Date"])
df3 = pd.merge(df2, nbr_doses_per_million_df, how="left", on=["Entity","Date"])
# df3.head(100)

In [4]:
# Clean up the merged file: 
# Remove duplicate data that is listed under Federal Agencies (in addition to being duplicates, they also contains many NaNs):
# Bureau of Prisons, Dept of Defense, Indian Health Svc, Long Term Care, Veterans Health
vaccinations_df = df3.loc[(df3["Entity"] != "Bureau of Prisons") &
                          (df3["Entity"] != "Dept of Defense") &
                          (df3["Entity"] != "Indian Health Svc") &
                          (df3["Entity"] != "Long Term Care") &
                          (df3["Entity"] != "Veterans Health"), :].copy()

# Replace remaining NaN values with zeros - these primarily occurred on the first day of data collection for some states.
vaccinations_df.fillna(value=0, inplace=True)

# Change the columns back to ints (fillna added a decimal position)
vaccinations_df['daily_vaccinations'] = vaccinations_df['daily_vaccinations'].astype(int) 
vaccinations_df['daily_vaccinations_per_million'] = vaccinations_df['daily_vaccinations_per_million'].astype(int)


In [5]:
# Restructure the data before outputting it
# Change column name from Entity to State to better reflect the content of the final, cleaned up dataframe. 
vaccinations_df.rename(columns={'Entity':'State'}, 
                 inplace=True)

# Remove the rows of national (state="US") data into its own csv for ease of creating the 2 tables US_vaccinations
# and State_vaccinations.
# Important note: the national and state numbers aren't always the same, because of the way that the different
# jurisdictions report their data and how the CDC cross-checks and totals it up
US_vaccinations_df = vaccinations_df.loc[(vaccinations_df["State"] == "United States"), :].copy()
US_vaccinations_df.reset_index(drop=True, inplace=True)
state_vaccinations_df = vaccinations_df.loc[(vaccinations_df["State"] != "United States"), :].copy()
state_vaccinations_df.reset_index(drop=True, inplace=True)


In [8]:
# Write the merged/cleaned up files to new csv file
US_vaccinations_df.to_csv(r"Resources\US_vaccinations.csv", index = False, encoding="utf-8")
state_vaccinations_df.to_csv(r"Resources\State_Vaccinations.csv", index = False, encoding="utf-8")

In [7]:
US_vaccinations_df.head(100)

,State,Date,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,daily_vaccinations_per_million
0,United States,1/12/2021,782228,0.24,641524,1932
1,United States,1/13/2021,1020260,0.31,710238,2139
2,United States,1/15/2021,1610524,0.49,798707,2406
3,United States,1/19/2021,2023124,0.61,911493,2745
4,United States,1/20/2021,2161419,0.65,892403,2688
5,United States,1/21/2021,2394961,0.72,913912,2753
6,United States,1/22/2021,2756953,0.83,975540,2938
7,United States,1/23/2021,3027865,0.91,1057387,3185
8,United States,1/24/2021,3216836,0.97,1122182,3380
9,United States,1/25/2021,3346390,1.01,1126251,3392
